In [1]:
from dotenv import load_dotenv
from pydantic import BaseModel
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from langchain.agents import create_tool_calling_agent, AgentExecutor
from tools import duckduckgo_tool, wiki_tool, save_tool

In [2]:
load_dotenv()

True

In [3]:
llm = ChatOpenAI(model="gpt-4o", temperature=0.3, max_tokens=1000)
response = llm.invoke("What is John Mayer's three most popular song? - Answer the title only. Separated with a comma.")
print(response.content)

"Gravity," "Slow Dancing in a Burning Room," "Your Body Is a Wonderland"


In [4]:
llm = ChatOpenAI(model="gpt-4o", temperature=0.3, max_tokens=1000)

class ResearchResponse(BaseModel):
    title: str
    summary: str
    sources: list[str]
    tools_used: list[str]

parser = PydanticOutputParser(pydantic_object=ResearchResponse)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system",
         """
        You are a research assitant that will help generate a research paper.
        Answer the user query and use neccessary tools.
        Wrap the output in this format and provide no other text\n{format_instructions}
         """
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{query}"),
        ("placeholder", "{agent_scratchpad}")

    ]
).partial(format_instructions=parser.get_format_instructions())

agent = create_tool_calling_agent(
    llm=llm,
    prompt=prompt,
    tools = []
)

# AgentExecutor will automatically handle chat_history and agent_scratchpad
executor = AgentExecutor(agent=agent, tools=[], verbose=True) # verbose is to see the agent's thought process

raw_response = executor.invoke({"query": "What is John Mayer's most popular song?"})
print(raw_response)





> Entering new AgentExecutor chain...
{"title": "John Mayer's Most Popular Song", "summary": "John Mayer's most popular song is often considered to be \"Gravity.\" This song, released as part of his album \"Continuum\" in 2006, has received widespread acclaim and is frequently highlighted in his live performances. It showcases Mayer's guitar skills and has become a staple in his music repertoire.", "sources": ["https://www.billboard.com/music/john-mayer"], "tools_used": []}

> Finished chain.
{'query': "What is John Mayer's most popular song?", 'output': '{"title": "John Mayer\'s Most Popular Song", "summary": "John Mayer\'s most popular song is often considered to be \\"Gravity.\\" This song, released as part of his album \\"Continuum\\" in 2006, has received widespread acclaim and is frequently highlighted in his live performances. It showcases Mayer\'s guitar skills and has become a staple in his music repertoire.", "sources": ["https://www.billboard.com/music/john-mayer"], "tools

In [5]:
llm = ChatOpenAI(model="gpt-4o", temperature=0.3)

class ResearchResponse(BaseModel):
    title: str
    summary: str
    sources: list[str]
    tools_used: list[str]

parser = PydanticOutputParser(pydantic_object=ResearchResponse)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system",
         """
        You are a research assitant that will help generate a research paper.
        Answer the user query and use neccessary tools.
        Wrap the output in this format and provide no other text\n{format_instructions}
         """
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{query}"),
        ("placeholder", "{agent_scratchpad}")

    ]
).partial(format_instructions=parser.get_format_instructions())

tools = [duckduckgo_tool, wiki_tool, save_tool]
agent = create_tool_calling_agent(
    llm=llm,
    prompt=prompt,
    tools = tools
)

# AgentExecutor will automatically handle chat_history and agent_scratchpad
executor = AgentExecutor(agent=agent, tools=tools, verbose=True) # verbose is to see the agent's thought process
query = input("What can I help you research?")
raw_response = executor.invoke({"query": query})

print(raw_response)



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'John Mayer'}`


Page: John Mayer
Summary: John Clayton Mayer ( MAY-ər; born October 16, 1977) is an American singer,
Invoking: `DuckDuckGo` with `John Mayer most famous songs`


John 起源于《新约圣经》里的人物 John the Baptist （基督教和合本译为 施洗约翰，天主教译为 圣若翰洗者）。施洗约翰 … 为什么「约翰」成为了 John 的主流译名？ 其实情况和「大卫」「彼得」类似，这些和《圣经》有关的名字，在规范的译音标 … john是什么意思？john有两种意思，一种是作为名词，有几种含义盥洗室,厕所，男人；傻瓜，糊涂虫，嫖客，警察等；另一 … Name: John Doe Jr. First Name: John Last Name: Doe Suffix: Jr. - - - - - If you only see a single name and you're not sure … first name 是名字，last name是姓。Mary, John是一个人的名字，他们经常把名放前面（first name ），而姓放后面 (last …

/Users/junicso/250724-demo-AI-agent/venv/lib/python3.11/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:



Invoking: `save_text_to_file` with `John Mayer is an American singer-songwriter known for his blues-inspired guitar playing and smooth vocals. Some of his most famous songs include "Your Body Is a Wonderland," "Gravity," "Slow Dancing in a Burning Room," "Waiting on the World to Change," and "Daughters." Mayer has won several Grammy Awards and is recognized for his contributions to contemporary music.`


Data saved to research_output.txt```json
{
    "title": "John Mayer's Most Famous Songs",
    "summary": "John Mayer is an American singer-songwriter known for his blues-inspired guitar playing and smooth vocals. Some of his most famous songs include \"Your Body Is a Wonderland,\" \"Gravity,\" \"Slow Dancing in a Burning Room,\" \"Waiting on the World to Change,\" and \"Daughters.\" Mayer has won several Grammy Awards and is recognized for his contributions to contemporary music.",
    "sources": [
        "Wikipedia",
        "DuckDuckGo"
    ],
    "tools_used": [
        "functions